<a href="https://colab.research.google.com/github/geovaneBausen/dados/blob/main/youtubefrag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Este projeto expande o poder de análise do Gemini para vídeos, superando suas limitações de tamanho e download.
 Os destaques são:

*Análise de vídeos em larga escala: Processa vídeos de qualquer tamanho hospedados no YouTube.


*Integração com o YouTube: Baixa vídeos e extrai transcrições automaticamente via API.


*Resumo automático com Gemini: Usa o Gemini para gerar resumos concisos das transcrições.


*Visualização completa: Apresenta resumos e timelines dos vídeos processados de forma clar




In [1]:
#Modo de Depuração (Debug Mode)
#A função debug_print foi criada para imprimir mensagens de depuração apenas quando o modo de depuração está ativado. Para ativar o modo de depuração, defina a variável debug como True

debug = False

def debug_print(message):
    if debug:
        print(message)

In [2]:
!pip install -q -U google -generativeai


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -g


In [3]:
!pip install pytube youtube_transcript_api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.7 MB/s eta 0:00:00


In [21]:
#  URL do vídeo que você quer baixar
video_url = "https://www.youtube.com/watch?v=OmewFkKuidg"

from pytube import YouTube
from youtube_transcript_api import YouTubeTranscriptApi
import json

try:
    yt = YouTube(video_url)
    video_stream = yt.streams.get_highest_resolution()

    # Download do vídeo
    video_stream.download()

    # Extrair o ID do vídeo ID from the URL
    video_id = video_url.split("v=")[1]

    # Transcript do vídeo
    transcripts = YouTubeTranscriptApi.get_transcript(video_id, languages=['pt', 'en'])

    if transcripts:
        subtitle_text = " ".join([entry["text"] for entry in transcripts])

        debug_print("*** Transcripts:")
        debug_print(subtitle_text)
    else:
        debug_print("Transcript não disponível para este vídeo.")

except Exception as e:
    debug_print(f"Erro: {str(e)}")


In [22]:
def split_transcript(transcript, max_duration=600):
    parts = []
    current_part = []
    current_duration = 0

    for entry in transcript:
        if current_duration + entry["duration"] <= max_duration:
            # Serializa a entrada para JSON e adiciona à parte atual
            current_part.append(json.dumps(entry))
            current_duration += entry["duration"]
        else:
            # Decodifica cada entrada de transcrição na parte atual
            decoded_part = [bytes(entry, "utf-8").decode("unicode_escape") for entry in current_part]
            parts.append(decoded_part)  # Adiciona a parte atual à lista de partes
            current_part = [json.dumps(entry)]  # Começa uma nova parte com a entrada atual
            current_duration = entry["duration"]

    # Decodifica a última parte
    decoded_part = [bytes(entry, "utf-8").decode("unicode_escape") for entry in current_part]
    parts.append(decoded_part)

    return parts


# Dividir o transcript em partes
segments = split_transcript(transcripts, max_duration=600)

debug_print (f"Transcript dividido em {len(segments)} partes")
for segment in segments:
  debug_print (f"{segment[:1000]}")
  debug_print ("---")

In [23]:
import google.generativeai as genai
GOOGLE_API_KEY="'-''"
genai.configure(api_key=GOOGLE_API_KEY)

In [24]:

generation_config = {
    "candidate_count" : 1,
    "temperature" : 0.5
}

safety_settings = {
    "HARASSMENT" : "BLOCK_NONE",
    "HATE" : "BLOCK_NONE",
    "SEXUAL" : "BLOCK_NONE",
    "DANGEROUS" : "BLOCK_NONE"
}

model = genai.GenerativeModel("gemini-1.0-pro",
                              generation_config = generation_config,
                              safety_settings = safety_settings)


In [25]:
prompt = "Preciso descrever o conteúdo de um vídeo do YouTube."
prompt += "\n".join(segments[0])

debug_print(prompt)

In [26]:
summaries = []
clips = []

for segment in segments:
  prompt = "Preciso descrever o conteúdo de um vídeo do YouTube."
  prompt += " Aqui está o conteúdo:"
  prompt += "\n".join(segment)
  prompt += " Os dados do vídeo estão no formato JSON {text:'', start:'', duration:''}. start representa o início do trecho em segundos, e duration representa a duração do trecho em segundos;"
  prompt += " Quais são os assuntos discutidos nesse vídeo? Crie uma lista contendo uma descrição do assunto e o valor do campo start em que se inicia o assunto."
  prompt += " Também preciso que você faça um breve resumo do vídeo inteiro."
  prompt += " Responda no formato JSON, de forma que uma aplicação possa fazer o parse da resposta. Os campos do JSON devem ser exatamente estes: 1. summary: resumo do vídeo e 2: topics: lista dos assuntos, que também deve ter um formato JSON assim definido: 1. description e 2.start"
  prompt += " O valor de start deve ser exatamente igual ao encontrado no json original."
  response = model.generate_content(prompt)
  response_text = response.text
  response_text = response_text.replace("```json", '')
  response_text = response_text.replace("```", '')

  json_object = json.loads(response_text)

  summaries.append(json_object["summary"])
  clips.extend(json_object["topics"])

for clip in clips:
    debug_print(clip)

In [27]:
summaries_string = '\n'.join(summaries)

prompt = "Preciso descrever o conteúdo de um vídeo do YouTube."
prompt += " O vídeo contém vários trechos. Vou enviar o resumo de cada trecho. Você deve ler todos eles e gerar um resumo geral."
prompt += " Retorne apenas o resumo geral. Estes são os resumos:\n"

prompt += summaries_string
response = model.generate_content(prompt)

total_summary = response.text

In [28]:
import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

display (to_markdown("### Resumo:"))
display (to_markdown(total_summary))
display (to_markdown("### Timeline:"))

for clip in clips:
  if isinstance(clip['start'], str):
    clip['start'] = clip['start'].replace("'", "")
    clip['start'] = float(clip['start'])

for clip in clips:
    link = f"{video_url}&t={clip['start']}s"
    hours = int(clip['start'] // 3600)
    minutes = int((clip['start'] % 3600) // 60)
    seconds = int(clip['start'] % 60)

    # Formato do tempo HH:MM:SS
    time_formatted = f"{hours:02d}:{minutes:02d}:{seconds:02d}"
    #time_formatted = clip['start']
    start_aligned = f"{time_formatted}".ljust(10)
    description_aligned = clip['description'].ljust(50)
    display (to_markdown(f"{start_aligned} - [{description_aligned}]({link})"))

> ### Resumo:

> O vídeo apresenta uma música inspiradora que incentiva os espectadores a superar desafios, buscar o sucesso e encontrar força em suas comunidades. As letras motivacionais são acompanhadas por imagens de indivíduos em ambientes urbanos, destacando a jornada pessoal e a importância de conexões interpessoais. O vídeo transmite uma mensagem de resiliência, determinação e o poder de apoio para alcançar objetivos.

> ### Timeline:

> 00:00:28   - [Motivação e inspiração                            ](https://www.youtube.com/watch?v=OmewFkKuidg&t=28.96s)

> 00:00:35   - [Superação de desafios                             ](https://www.youtube.com/watch?v=OmewFkKuidg&t=35.879s)

> 00:00:39   - [Importância da comunidade                         ](https://www.youtube.com/watch?v=OmewFkKuidg&t=39.36s)

> 00:00:54   - [Busca pelo sucesso                                ](https://www.youtube.com/watch?v=OmewFkKuidg&t=54.719s)

> 00:01:09   - [Igualdade e justiça social                        ](https://www.youtube.com/watch?v=OmewFkKuidg&t=69.4s)

> 00:01:27   - [Autoaperfeiçoamento e educação                    ](https://www.youtube.com/watch?v=OmewFkKuidg&t=87.479s)

> 00:02:14   - [Esperança e otimismo                              ](https://www.youtube.com/watch?v=OmewFkKuidg&t=134.879s)